In [29]:
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [18]:
df1=pd.read_csv("model_df", na_values =['NaN'])
df1.index

RangeIndex(start=0, stop=55250, step=1)

In [19]:
df1=df1.dropna()
df1.index

Int64Index([    1,     7,    10,    13,    16,    20,    27,    28,    29,
               30,
            ...
            55236, 55237, 55239, 55240, 55241, 55242, 55243, 55244, 55245,
            55247],
           dtype='int64', length=20543)

In [21]:
X=df1.drop(['Fall', 'Unnamed: 0'], axis=1)
# .values
y=df1['Fall'].values
# print(X.shape, y.shape)
X.columns

Index(['Difficulty', 'Event', 'Rider_x', 'Horse_x', 'Location', 'Senior',
       'Junior', 'CT', 'Dress Score', 'Dress Place', 'precip-end',
       'Precip_Prior_3_Day', 'Precip_Prior_5_Day'],
      dtype='object')

In [22]:
X=X.values

In [23]:
label_encoder = LabelEncoder()
label_encoder.fit(X[:, 1])
X_encoded_Event=label_encoder.transform(X[:, 1])
decoder=label_encoder

label_encoder = LabelEncoder()
label_encoder.fit(X[:, 2])
X_encoded_Rider_x=label_encoder.transform(X[:, 2])

label_encoder = LabelEncoder()
label_encoder.fit(X[:, 3])
X_encoded_Horse_x=label_encoder.transform(X[:, 3])

label_encoder = LabelEncoder()
label_encoder.fit(X[:, 4])
X_encoded_Location=label_encoder.transform(X[:, 4])

In [24]:
X_encoded = np.c_[X[:, 0],
                  X_encoded_Event,
                  X_encoded_Rider_x,
#                   X_encoded_Horse_x,
                  X_encoded_Location,
                  X[:, 5:]]
X_encoded.shape

(20543, 12)

In [25]:
scaler=MinMaxScaler().fit(X[:, 8:])
X_scaler=scaler.transform(X[:, 8:])
X_scaler.shape

(20543, 5)

In [26]:
X_categorical_Difficulty=to_categorical(X[:, 0])
X_categorical_Senior=to_categorical(X[:, 5])
X_categorical_Junior=to_categorical(X[:, 6])
X_categorical_CT=to_categorical(X[:, 7])
X_categorical_Event=to_categorical(X_encoded_Event)
X_categorical_Rider_x=to_categorical(X_encoded_Rider_x)
X_categorical_Horse_x=to_categorical(X_encoded_Horse_x)
X_categorical_Location=to_categorical(X_encoded_Location)
X_categorical_Event.shape

(20543, 132)

In [27]:
X_categorical = np.c_[X_categorical_Event,
                      X_categorical_Difficulty,
                      X_categorical_Senior,
                      X_categorical_Junior,
                      X_categorical_CT,
                      X_categorical_Rider_x,
#                       X_categorical_Horse_x,
                      X_categorical_Location,
                      X_scaler]

In [30]:
X_train, X_test, y_train, y_test = train_test_split(
    X_categorical, y, random_state=1)

In [31]:
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [ ]:
# Loop through different k values to see which has the highest accuracy
# Note: We only use odd numbers because we don't want any ties
train_scores = []
test_scores = []
for k in range(1, 20, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train_categorical)
    train_score = knn.score(X_train, y_train_categorical)
    test_score = knn.score(X_test, y_test_categorical)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")
    
    
plt.plot(range(1, 20, 2), train_scores, marker='o')
plt.plot(range(1, 20, 2), test_scores, marker="x")
plt.xlabel("k neighbors")
plt.ylabel("Testing accuracy Score")
plt.show()

k: 1, Train/Test Score: 1.000/0.948
k: 3, Train/Test Score: 0.974/0.971
k: 5, Train/Test Score: 0.973/0.972


In [ ]:
knn = KNeighborsClassifier(n_neighbors=9)
knn.fit(X_train, y_train_encoded)
print('k=9 Test Acc: %.3f' % knn.score(X_test, y_test_categorical))

In [ ]:
predicted_fall = knn.predict(y_test_categorical)